In [1]:
import json
import pandas as pd
import os

In [7]:
# === Paths ===
json_input_path = r"C:\Users\1176153\Downloads\github\Thesis\data\Preprocessing_text\generation_evaluation_dataset.json"
ref_xlsx_path = r"C:\Users\1176153\Downloads\github\Thesis\data\Preprocessing_text\ref_data.xlsx"
gen_xlsx_path = r"C:\Users\1176153\Downloads\github\Thesis\model\src\rag_no_prompting_medium_Mistral_results.xlsx"
final_output_json_path = r"C:\Users\1176153\Downloads\github\Thesis\data\Preprocessing_text\evaluation_generation_dataset_mistral_hard.json"

### === STEP 1: Extract from JSON to XLSX ===

with open(json_input_path, "r", encoding="utf-8") as f:
    data = json.load(f)

# Grab first 29 items only
records = data["evaluation_generation_dataset"][:29]

ref_data = {
    "reference_answer": [entry["reference_answer"][0] if entry["reference_answer"] else "" for entry in records],
    "relevant_docs": [", ".join(entry["relevant_docs"]) for entry in records]
}

ref_df = pd.DataFrame(ref_data)
ref_df.to_excel(ref_xlsx_path, index=False)
print(f"✅ Extracted reference data saved to {ref_xlsx_path}")

✅ Extracted reference data saved to C:\Users\1176153\Downloads\github\Thesis\data\Preprocessing_text\ref_data.xlsx


In [8]:
### === STEP 2: Recombine all into final JSON ===

# Load 50 question-generated_answer pairs
gen_df = pd.read_excel(gen_xlsx_path, engine="openpyxl", skiprows=1, usecols=[0, 1], nrows=50)
gen_df.columns = ["question", "generated_answer"]
gen_df.fillna("", inplace=True)

# Load reference answers and docs (29 rows)
ref_df = pd.read_excel(ref_xlsx_path)
ref_df.fillna("", inplace=True)

final_entries = []

for idx in range(len(gen_df)):
    question = gen_df.loc[idx, "question"]
    generated_answer = gen_df.loc[idx, "generated_answer"]

    if idx < len(ref_df):
        reference_answer = [ref_df.loc[idx, "reference_answer"]]
        relevant_docs = [doc.strip() for doc in str(ref_df.loc[idx, "relevant_docs"]).split(",")]
    else:
        reference_answer = [""]
        relevant_docs = [""]

    entry = {
        "id": idx + 1,
        "question": question,
        "relevant_docs": relevant_docs,
        "reference_answer": reference_answer,
        "generated_answer": [generated_answer],
        "human_score": 0
    }
    final_entries.append(entry)

# Save to final JSON
with open(final_output_json_path, "w", encoding="utf-8") as f:
    json.dump({"evaluation_generation_dataset_mistral_hard": final_entries}, f, indent=2, ensure_ascii=False)

print(f"✅ Final JSON file with 50 entries saved to: {final_output_json_path}")

✅ Final JSON file with 50 entries saved to: C:\Users\1176153\Downloads\github\Thesis\data\Preprocessing_text\evaluation_generation_dataset_mistral_hard.json
